### 손글씨 이미지 데이터 MNIST

- 인코딩된 바이너리 데이터를 디코딩하여 처리하는 방식 확인
- 지도학습
- 학습용 데이터는 6만개, 테스트 데이터는 1만개
- 결론 : 
    - 학습후 새로운 데이터를 입력시 판별
    - 0 ~ 9 까지의 손글씨 이미지를 판별
    - http://yann.lecun.com/exdb/mnist/index.html
    - 데이터는 URL 을 직접 획득해서, 원하는 곳에 다운로드

### 절차

|No|단계|내용|
|:---:|:---|:---|
|1|연구목표|- 손글씨 이미지(0-9)를 학습시켜서 새로운 손글씨 이미지를 판별해 내는 머신러닝 모델을 구축<br>- 압축된 이미지를 압축해제<br>- 인코딩 된 데이터를 디코딩 처리<br>- 28X28로 구성된 픽셀 이미지데이터를 벡터화 처리<br>- 시스템 통합의 결과를 보고 연구목표를 설정해야하는데 시스템 통합을 생략하므로 이부분은 생략한다.|
|2|데이터획득/수집|- http://yann.lecun.com/exdb/mnist 접속<br>- Web Scaraping을 통해서 데이터의 URL 획득 <br>- 지정된 위치에 다운로드 -> 압축해제|
|3|데이터준비/전처리|- 디코딩(내부구조를 알 수 있는 인코딩 문서(MNIST Database) 필요)<br>- 엔디안(Endian)처리(추가적인 내용)<br>- 벡터화 처리|
|4|데이터탐색/통찰/시각화분석|- skip (제공된 데이터가 명확함)|
|5|데이터 모델링(머신러닝모델)|- 분류 알고리즘 사용<br>- 알고리즘선택 -> 훈련용,학습용 데이터로 나눔 -> 학습 -> 예측 ->평가|
|6|시스템통합|- skip|

#### 2. 데이터 획득/ 수집
- 모듈준비

In [8]:
import urllib.request as req
from bs4 import BeautifulSoup

- web scraping

In [9]:
rootUrl = 'http://yann.lecun.com/exdb/mnist/'
soup    = BeautifulSoup(req.urlopen(rootUrl), 'html5lib')

- train-images-idx3-ubyte.gz ,...등 총 4개의 url 획득

In [10]:
# 모든 요소 tt 중에 상위 4개만 링크
for tt in soup.findAll('tt')[:4]:
    # 링크 값이나 링크 문자열이나 현재 동일하기 때문에 문자열 획득으로 처리
    # 링크 최종주소는 rootUrl + 링크 문자열을 더한것이다.
    print(tt.a.string)  # 또는 print(tt.a.text)

train-images-idx3-ubyte.gz
train-labels-idx1-ubyte.gz
t10k-images-idx3-ubyte.gz
t10k-labels-idx1-ubyte.gz


In [11]:
# 링크 문자열을 리스트에 담은 이유는 반복 작업이 예상 되었기 때문이다.
files = [ tt.a.string for tt in soup.findAll('tt')[:4] ] # 리스트내포
files

['train-images-idx3-ubyte.gz',
 'train-labels-idx1-ubyte.gz',
 't10k-images-idx3-ubyte.gz',
 't10k-labels-idx1-ubyte.gz']

- 다운로드 > 압축해제 <= 반복작업 총 4번 수행

In [12]:
# 필요 모듈
import os, os.path, gzip

In [13]:
# 위치 선정(압축된 파일을 다운로드 할 위치)
savePath = './data/mnist'
# 해당 디렉토리가 없으면 만들도록 하기
if not os.path.exists(savePath):  # 물리적으로 해당 경로가 없다.
    os.makedirs(savePath)

In [14]:
# 저장
# tqdm : 진행율을 보여주는 모듈
from tqdm import tqdm_notebook
for file in tqdm_notebook(files):
    
    print( '소스',rootUrl + file )
    
    # 저장위치 및 파일명
    local_path = '%s/%s' % (savePath , file)
    print( '대상',local_path )
    
    # 웹상에 존재하는 리소스를 로컬 디스크 상에 직접 저장
    req.urlretrieve( rootUrl + file, local_path )

소스 http://yann.lecun.com/exdb/mnist/train-images-idx3-ubyte.gz
대상 ./data/mnist/train-images-idx3-ubyte.gz
소스 http://yann.lecun.com/exdb/mnist/train-labels-idx1-ubyte.gz
대상 ./data/mnist/train-labels-idx1-ubyte.gz
소스 http://yann.lecun.com/exdb/mnist/t10k-images-idx3-ubyte.gz
대상 ./data/mnist/t10k-images-idx3-ubyte.gz
소스 http://yann.lecun.com/exdb/mnist/t10k-labels-idx1-ubyte.gz
대상 ./data/mnist/t10k-labels-idx1-ubyte.gz



In [15]:
# 압축해제
# 원본 : train-images-idx3-ubyte.gz
# 해제 : train-images-idx3-ubyte  <= 원본 파일의 이름을 사용하겠다.
for file in tqdm_notebook(files):
    # 원본 파일의 경로
    ori_gzip_file = '%s/%s' % (savePath , file)
    print( ori_gzip_file )
    # 압축 해제 파일의 경로 , 파일명은 다양한 방법으로 획득 가능
    # 파일명에 반드시 확장자가 있을 필요는 없다
    target_file ='%s/%s' % (savePath , file[:-3]) 
    print( target_file )
    # 압축해제 
    # gzip의 파일오픈 -> 읽기'rb' -> 쓰기
    with gzip.open(ori_gzip_file, 'rb' ) as fg:
        # 읽기 => 압축해제를 수행했다
        tmp = fg.read()
        # 쓰기 => 일반파일로 기록
        with open( target_file, 'wb' ) as f:
            f.write(tmp)

./data/mnist/train-images-idx3-ubyte.gz
./data/mnist/train-images-idx3-ubyte
./data/mnist/train-labels-idx1-ubyte.gz
./data/mnist/train-labels-idx1-ubyte
./data/mnist/t10k-images-idx3-ubyte.gz
./data/mnist/t10k-images-idx3-ubyte
./data/mnist/t10k-labels-idx1-ubyte.gz
./data/mnist/t10k-labels-idx1-ubyte



#### 3. 데이터준비/전처리
- 디코딩(내부구조를 알 수 있는 인코딩 문서(MNIST Database) 필요)
- 엔디안(Endian)처리(TCP/IP 상에서 통신 수행시 중요)
    - 컴퓨터 메모리와 같은 1차원 공간에 여러개의 연속된 데이터를 배열하는 방법
    - 종류 : 바이트를 배치하는 오더(순서)를 앞에서부터 혹은 뒤에서부터 채우는가
        - 0x12345678
        - 빅 에디언
            - 낮은(시작)주소에 상위 바이트부터 기록, Sparc / RISC CPU 계열
            - 0x12 0x34 0x56 0x78  
        - 리틀 에디언
            - 낮은(시작) 주소에 하위 바이트부터 기록, Intel CPU 계열
            - 0x78 0x56 0x34 0x12
        - 위의 예는 정수값 (4 byte)을 예를 든것이고, 단지 값이 어떻게 기록됐는지만 이해하고, 그대로 값을 복원할 수 있으면 끝    
- 벡터화 처리

- LABEL FILE
    - magic number : 4byte (32bit)  => 엔디안 체크
    - LABEL 수 : 4byte              => 엔디안 체크
    - LABEL 데이터 : 1byte ...      => 0 ~ 9 값
    - 크기 = 4 + 4 + LABEL수 * 1byte = 4 + 4 + 60000 = 60008 byte
    
|offset|type|value|description|
|:---:|:---|:---|:---|
|0000|32 bit integer|0x00000801(2049)|magic number (MSB first)|
|0004|32 bit integer|60000|number of items|
|0008|unsigned byte|??|label|


- IMAGE FILE
    - magic number : 4byte (32bit)      => 엔디안 체크
    - 손글씨 이미지 개수 : 4byte        => 엔디안 체크
    - 가로크기(픽셀수) : 4byte          => 엔디안 체크
    - 세로크기(픽셀수) : 4byte          => 엔디안 체크
    - 픽셀값 한개 한개씩 : unsigned 1 byte(=8 bit) (0 ~ 2^8- 1 : 0~255(0xFF))
    - 크기 
    
|offset|type|value|description|
|:---:|:---|:---|:---|
|0000|32 bit integer|0x00000801(2051)|magic number (MSB first)|
|0004|32 bit integer|60000|number of images|
|0008|32 bit integer|28|number of rows|   
|0012|32 bit integer|28|number of columns| 
|0016|unsigned byte|??|pixel| 


In [16]:
# 원데이터의 구조를 
import struct
# struct : 바이너리 데이터를 빅/리틀 엔디안 방식으로 특정 바이트만큼 읽는 기능을 제공한다.

In [17]:
# 테스트용 레이블 파일 처리
# 바이너리 읽기 모드
label_f = open('./data/mnist/t10k-labels-idx1-ubyte', 'rb')

# 바이너리 데이터는 헤더부터 읽어서 데이터의 유효성이나 종류를 인지
# MNIST 파일은 규격서에 high(빅) endian 으로 수치값을 기술했다고 확인했다.
# label 파high(빅) endian일은 헤더가  4 + 4 + 8 byte 이다 (규격서 기준)
# high(빅) endian => >
# 4 -> I (i 의 대문자)
magic_number, label_count = struct.unpack( '>II' , label_f.read(8) )

# magic_number : 2049 레이블 파일이다
# label_count  : 데이터의 개수 (레이블의 개수, 답의 개수)
print(magic_number, label_count)

# 닫기
label_f.close()

2049 10000


In [18]:
# 테스트용 이미지파일 처리 - 위쪽 참고
image_f = open('./data/mnist/t10k-images-idx3-ubyte', 'rb')

magic_number2, label_count, row, col = struct.unpack( '>IIII' , image_f.read(16) )

print(magic_number2, label_count, row, col)

image_f.close()

2051 10000 28 28


In [19]:
# 헤더 정보 추출
# 헤더를 분석하면 바디를 알 수 있다
label_f = open('./data/mnist/t10k-labels-idx1-ubyte', 'rb')
image_f = open('./data/mnist/t10k-images-idx3-ubyte', 'rb')

magic_number, label_count = struct.unpack( '>II' , label_f.read(8) )
print(magic_number, label_count)

magic_number2, image_count, row, col = struct.unpack( '>IIII' , image_f.read(16) )
print(magic_number2, image_count, row, col)

# 헤더크기 = 16 + 이미지데이터크기(28*28) * 이미지 총개수(10000)
print('이미지 파일의 크기', 4 + 4 + 4 + 4 + 10000*28*28, 'bytes')

# 헤더 정보를 기초로 반복 작업 수행 : 정답추출, 이미지 추출
for idx in range(image_count) : # label_count 사용해도 동일
    # 정답추출 : label_f를 통해서 1 byte 읽는다. 
    # 단, unsigned(부호없는,양수) byte -> 'B'
    # 파일을 읽으면 읽은만큼 누적으로 커서(파일포인터) 위치가 이동한다 
    label_tmp = struct.unpack( 'B', label_f.read(1) ) 
    #(7,) 이렇게 리턴 : 손글씨 숫자 7 -> 인덱싱을 통해서 값 획득
    label = label_tmp[0]
    # print(label) 
    
    # 이미지 추출
    
    
    
    
    
    # 이미지 데이터의 벡터화 처리
    break
    pass

label_f.close()
image_f.close()

2049 10000
2051 10000 28 28
이미지 파일의 크기 7840016 bytes


In [20]:
# 코드, 주석이 길어지면 다음칸으로 옮김
label_f = open('./data/mnist/t10k-labels-idx1-ubyte', 'rb')
image_f = open('./data/mnist/t10k-images-idx3-ubyte', 'rb')

magic_number, label_count = struct.unpack( '>II' , label_f.read(8) )
magic_number2, image_count, row, col = struct.unpack( '>IIII' , image_f.read(16) )

# 이미지 한개당 크기 28*28
pixels = row*col # 28*28
for idx in range(image_count) :
    label_tmp = struct.unpack( 'B', label_f.read(1) ) 
    label = label_tmp[0]    
    # 이미지 추출 ->  바이너리 데이터를 읽는다 -> 엔디안은 관계없음
    binarryData = image_f.read(pixels)
    # print(type(binarryData),len(binarryData) ,binarryData)
    # 픽셀값 하나 하나를 문자열로 만들어서 리스트에 담는다
    # 바이너리 값은 0~255의 문자열로 변경했다
    strData = list( map( lambda x:str(x) , binarryData ) )
    # print(strData)
    
    # csv에 한줄의 데이터로 기록 => 1 + 784 개를 기록 => 1개의 데이터를 표현 
    # 구분자 ,  
    
    # pgm 파일로 dump 처리해서 확인 (데이터의 유효성 확인)
    
    
    
    
    
    # 이미지 데이터의 벡터화 처리
    break
    pass

label_f.close()
image_f.close()

In [21]:
label_f = open('./data/mnist/t10k-labels-idx1-ubyte', 'rb')
image_f = open('./data/mnist/t10k-images-idx3-ubyte', 'rb')
# csv 파일 오픈
csv_f   = open( './data/mnist/t10k.csv', 'w', encoding='utf-8' )


magic_number, label_count = struct.unpack( '>II' , label_f.read(8) )
magic_number2, image_count, row, col = struct.unpack( '>IIII' , image_f.read(16) )

pixels = row*col
for idx in range(image_count) :
    label_tmp = struct.unpack( 'B', label_f.read(1) ) 
    label = label_tmp[0]    
    binarryData = image_f.read(pixels)
    strData = list( map( lambda x:str(x) , binarryData ) )  
    # csv에 한줄의 데이터로 기록 => 1 + 784 개를 기록 => 1개의 데이터를 표현 
    # 구분자 : , 
    csv_f.write( str(label)+',' ) #답
    csv_f.write(','.join( strData ) + '\n') # \n 줄바꿈 # 28*28의 손글씨 이미지 데이터
    
    # pgm 파일로 dump 처리해서 확인 (데이터의 유효성 확인)
    # pgm 관련주소 : https://en.wikipedia.org/wiki/Netpbm#File_formats
    if idx == 0: # 한번만 수행된다 (확인용으로 해봄)
        with open('./data/mnist/%s.pgm' % label, 'w', encoding='utf-8') as f:
            f.write( 'P2 28 28 255\n' + ' '.join(strData) )
        
    
    
    # 이미지 데이터의 벡터화 처리
    break
    pass

label_f.close()
image_f.close()
csv_f.close() 

In [27]:
def decoding_mnist_rawData( dataStyle='train',maxCount=0 ):
    label_f = open('./data/mnist/%s-labels-idx1-ubyte' % dataStyle, 'rb')
    image_f = open('./data/mnist/%s-images-idx3-ubyte' % dataStyle, 'rb')
    csv_f   = open( './data/mnist/%s.csv' % dataStyle, 'w', encoding='utf-8' )

    magic_number, label_count = struct.unpack( '>II' , label_f.read(8) )
    magic_number2, image_count, row, col = struct.unpack( '>IIII' , image_f.read(16) )

    if maxCount > image_count:
        print('개수의 범위를 넘었습니다. 최소 %s 개 이내' % image_count)
        return
    elif maxCount == -1:
        maxCount = image_count
    elif maxCount < -1:
        maxCount = image_count        
        print('개수의 범위를 넘었습니다!!. 최소 %s 개 이내' % image_count)
        return
    
    pixels = row*col
    for idx in tqdm_notebook( range(maxCount) ) :
        if idx >= maxCount:break
        label_tmp = struct.unpack( 'B', label_f.read(1) ) 
        label = label_tmp[0]    
        binarryData = image_f.read(pixels)
        strData = list( map( lambda x:str(x) , binarryData ) )  

        csv_f.write( str(label)+',' )
        csv_f.write(','.join( strData ) + '\n')


    label_f.close()
    image_f.close()
    csv_f.close() 

In [28]:
# 훈련용 데이터는 750개 테스트용은 250개를 준비한다
decoding_mnist_rawData(maxCount=2100) # 훈련용
decoding_mnist_rawData(dataStyle='t10k',maxCount=700) #테스트용

#### [M1] 데이터 품질 향상

- 머신러닝 모델을 이용하여 예측시 정확도가 떨어지면 데이터의 품질, 양을 검토한다
- 정확도 96% 목표로 머신러닝 모델을 개선한다
    [사전조치]
    - 양을 점차적으로 늘린다
        - 데이터의 개수를 늘린다
    - 품질을 향상시킨다
        - 정규화 => 여러가지 사용후 교차 검증
        - 차후에 적용가능한 내용 :PCA 같은 비지도 학습의 차원축소(피쳐의수를 줄인다: 독립변수의 갯수, 컬럼수) => 직교행렬
    - 비율을 조정한다(훈련:테스트 = 75:25)
    [모델개선조치]
    - 알고리즘 교체
    - 하이퍼파라미터 튜닝
    - 파이프라인을 이용한 전처리기를 활용 (품질향상) 하여 향상
    - 이런 교차 검증법을 활용하여 성능향상을 도모한다.
    - 이런것들의 검증은 ROC 곡선, AUC값 등으로 확인할수도있고, 교차검증법의 결과로 확인가능하다

#### 데이터탐색/통찰/시각화분석

- SKIP

In [ ]:
# 다음 함수를 만든다. load_csv( dataType='train' )
# 현재 csv 파일은 : t10k.csv , train.csv
# 입력 데이터 : csv 파일명
# 출력 데이터 : { 'labels':[], 'images':[[ ]] }  labels => 답(0~9) ,images =>784개

In [29]:
def load_csv( dataType='train' ):
    # 1. 파일명 생성
    file_path = './data/mnist/%s.csv' % dataType
    # 2. 데이터를 담을 그릇 준비
    labels = list()
    images = list()    
    # 3.csv 파일 오픈
    with open(file_path, 'r') as f:
        # 4. 한줄씩 읽겠다. => 한줄이 데이터의 한 세트
        for line in f:
            # line 에 대해서 확인
            # print( type(line) ,line )
            tmp = line.split(',')
            # 5. 데이터를 그릇에 추가 => 데이터는 원래대로 int 형으로 복원처리한다
            labels.append( int(tmp[0]) )
            images.append(list( map( lambda x:int(x) , tmp[1:] ) ))
   
    return { 'labels':labels, 'images':images } 
    
train = load_csv()
test  = load_csv('t10k')    

In [30]:
print(min(train['images'][0]),max(train['images'][0]))
# 값이 거리가 너무 멀다 -> 학습효과가 떨어진다 -> 0~1사이의 값으로 재배치 하는것이 합리적이다
# 정규화처리 
# 1번) 0: 0 이고 255:1 으로 만들어주기 <- minmax 스케일러
# 2번) 값의 총개수 256 <- 범주형자료(분류형) 자료로 본다 
# 픽셀데이터가 앞뒤 영향을 받는 연속적 성향을 가졌는가? 독립적인 성향을 가졌는가 ?

0 255


In [31]:
# 정규화처리 lambda x:int(x)/256 해줘서 minmax 스케일링
def load_csv_ex( dataType='train' ):
    file_path = './data/mnist/%s.csv' % dataType
    labels = list()
    images = list()    
    with open(file_path, 'r') as f:
        for line in f:
            tmp = line.split(',')
            labels.append( int(tmp[0]) )
            images.append(list( map( lambda x:int(x)/256 , tmp[1:] ) ))
   
    return { 'labels':labels, 'images':images } 
    
train = load_csv_ex()
test  = load_csv_ex('t10k')    

#### 데이터모델링(머신러닝모델링)

- 지도학습 데이터 이므로, 정확도를 통해서 평가를 1차로 수행

In [32]:
# 1. 모듈 가져오기
from sklearn import svm, model_selection, metrics

In [33]:
# 2. 알고리즘 생성
SEED = 2020 # 난수 고정ㅇ을 통해서 svc가 내부적으로 사용하는 난수값의 패턴을 고정
clf = svm.SVC(random_state=SEED)  # 난수고정

In [34]:
# 3. 데이터 분류 (이미 위에서 완료)
len(train['images']), len(train['labels'])

(2100, 2100)

In [36]:
# 4. 학습
clf.fit(train['images'], train['labels'])

C:\Users\admin\Anaconda3\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
    kernel='rbf', max_iter=-1, probability=False, random_state=2020,
    shrinking=True, tol=0.001, verbose=False)

In [37]:
# 5. 예측
predict = clf.predict(test['images'])

In [55]:
predict

array([1, 1, 1, ..., 1, 1, 1])

In [38]:
# 6. 성능평가
metrics.accuracy_score(test['labels'], predict)

0.84

In [39]:
# 7. 오차행렬(혼동행렬)을 이용한 평가
clf_report = metrics.classification_report(test['labels'], predict)
print(clf_report) 

              precision    recall  f1-score   support

           0       0.92      0.93      0.93        60
           1       0.84      0.99      0.91        84
           2       0.82      0.79      0.81        77
           3       0.89      0.70      0.78        73
           4       0.86      0.85      0.86        74
           5       0.75      0.89      0.81        64
           6       0.91      0.84      0.87        61
           7       0.81      0.80      0.81        70
           8       0.86      0.70      0.77        63
           9       0.79      0.89      0.84        74

    accuracy                           0.84       700
   macro avg       0.85      0.84      0.84       700
weighted avg       0.84      0.84      0.84       700



## MinMaxScaler

In [27]:
from sklearn.preprocessing import MinMaxScaler

In [68]:
scaler_minmax       = MinMaxScaler()
y_train_norm2 = scaler_minmax.fit_transform(train['images'])

In [69]:
clf.fit(y_train_norm2, train['labels'])

C:\Users\admin\Anaconda3\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
    kernel='rbf', max_iter=-1, probability=False, random_state=None,
    shrinking=True, tol=0.001, verbose=False)

In [70]:
predict = clf.predict(scaler_minmax.fit_transform(test['images']))

In [71]:
metrics.accuracy_score(test['labels'], predict)

0.8832

## StandardScaler

In [21]:
from sklearn.preprocessing import StandardScaler

In [31]:
scaler       = StandardScaler()
y_train_norm = scaler.fit_transform(train['images'])

In [32]:
clf.fit(y_train_norm, train['labels'])

C:\Users\admin\Anaconda3\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
    kernel='rbf', max_iter=-1, probability=False, random_state=0,
    shrinking=True, tol=0.001, verbose=False)

In [47]:
predict = clf.predict(scaler.fit_transform(test['images']))

In [48]:
metrics.accuracy_score(test['labels'], predict)

0.9565

## pipeline

In [22]:
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier

In [23]:
clf2 = SVC()

In [24]:
pipe = Pipeline( [
    ('preprocessing', StandardScaler()),
    ('classifier', SVC())
                ])

In [25]:
param_grid = [
    {
        'classifier': [SVC()],
        'preprocessing': [StandardScaler()],
        'classifier__C':[0.001,0.01,0.1,1,10,100],
        'classifier__gamma':[0.001,0.01,0.1,1,10,100]
    }
]

In [26]:
grid = GridSearchCV(pipe,param_grid,cv=5)

In [27]:
grid.fit(train['images'],train['labels'] )

GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=Pipeline(memory=None,
                                steps=[('preprocessing',
                                        StandardScaler(copy=True,
                                                       with_mean=True,
                                                       with_std=True)),
                                       ('classifier',
                                        SVC(C=1.0, cache_size=200,
                                            class_weight=None, coef0=0.0,
                                            decision_function_shape='ovr',
                                            degree=3, gamma='auto_deprecated',
                                            kernel='rbf', max_iter=-1,
                                            probability=False,
                                            random_state=Non...
                                             degree=3, gamma=0.001,
                      

In [28]:
grid.best_score_

0.8785714285714286

In [30]:
grid.score(test['images'],test['labels'])

0.8971428571428571

In [35]:
# 실험단계를 한개의 process로 정리

# 75:25의 비율을 유지한다는 상황 => range(1,5)
res_ac_scores = list()
res_ac_reports = list()
for cnt in range(1,5):
    # 1.csv로 저장
    decoding_mnist_rawData(maxCount=7500*cnt) # 훈련용
    decoding_mnist_rawData(dataStyle='t10k',maxCount=2500*cnt) #테스트용

    # 2.csv 로드
    train = load_csv_ex()
    test  = load_csv_ex('t10k') 

    # 3.모델 생성및 학습 예측 평가
    SEED = 2020 # 난수 고정을 통해서 svc가 내부적으로 사용하는 난수값의 패턴을 고정
    clf = svm.SVC(random_state=SEED)  
    clf.fit(train['images'], train['labels'])
    predict  = clf.predict(test['images'])
    ac_score = metrics.accuracy_score(test['labels'], predict)
    # 정확도 추가
    res_ac_scores.append(ac_score)
    # 오차행렬 추가
    clf_report = metrics.classification_report(test['labels'], predict)
    res_ac_reports.append(clf_report) 

C:\Users\admin\Anaconda3\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


C:\Users\admin\Anaconda3\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


C:\Users\admin\Anaconda3\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


C:\Users\admin\Anaconda3\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


In [36]:
res_ac_scores

[0.8828, 0.901, 0.924, 0.9371]

In [40]:
print(res_ac_reports[3])

              precision    recall  f1-score   support

           0       0.96      0.98      0.97       980
           1       0.96      0.99      0.97      1135
           2       0.94      0.92      0.93      1032
           3       0.93      0.93      0.93      1010
           4       0.92      0.95      0.93       982
           5       0.92      0.91      0.91       892
           6       0.94      0.96      0.95       958
           7       0.95      0.92      0.94      1028
           8       0.93      0.91      0.92       974
           9       0.93      0.91      0.92      1009

    accuracy                           0.94     10000
   macro avg       0.94      0.94      0.94     10000
weighted avg       0.94      0.94      0.94     10000

